In [1]:
from Crypto.Hash import keccak

In [2]:
from Crypto.Cipher import AES

In [3]:
from Crypto.Util import Counter

In [4]:
from Crypto.Protocol.KDF import scrypt

In [5]:
from eth_keys import keys

In [6]:
import json

In [7]:
import uuid

In [8]:
# Generate a private key. It's just a random number with 256 bits

In [9]:
keccak_hash = keccak.new(digest_bits=256)

In [10]:
very_random_string = "Hello, World!"

In [11]:
keccak_hash.update(very_random_string.encode())

In [12]:
private_key = keccak_hash.digest()

In [13]:
# In practice, an Ethereum private key must be within a certain range. https://en.bitcoin.it/wiki/Private_key#Range_of_valid_ECDSA_private_keys

In [14]:
max_key = 'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364140'
assert int.from_bytes(private_key, 'big') <= int(max_key, 16)
assert int.from_bytes(private_key, 'big') >= 1

In [15]:
print("Private key is {} and of length {} bytes".format(private_key.hex(), len(private_key)))

Private key is acaf3289d7b601cbd114fb36c4d29c85bbfd5e133f14cb355c3fd8d99367964f and of length 32 bytes


In [16]:
# Let's define a passphrase

In [17]:
passphrase = "Very secure"

In [18]:
salt = bytearray.fromhex("99d37a47c7c9429c66976f643f386a61b78b97f3246adca89abe4245d2788407")

number_of_rounds = 262144

In [19]:
number_of_rounds = 16

In [20]:
# The passhphrase is then stretched to 32 bytes through N rounds of Key Derivation Function: in this case `scrypt`

In [21]:
encryption_key = scrypt(passphrase, salt, 32, N=number_of_rounds, r=8, p=1)

In [22]:
print("Full encryption key is {} and of length {} bytes".format(encryption_key.hex(), len(encryption_key)))

Full encryption key is 8c7c68e4ddfc357d805e2ebd76dac0da162da3a4ce9482dfc96319cfacdbbc6e and of length 32 bytes


In [23]:
# Now we are ready to encode our private key

In [24]:
# In this example, we'll be using 128-bit (i.e. 16 bytes) AES encrypytion - counter mode

In [25]:
# Our key needs to be shortened to 16 bytes then. We keep the second half for validation purposes

In [26]:
validation_key = encryption_key[16:]
encryption_key = encryption_key[:16]

In [27]:
# Create the counter object

In [28]:
iv = bytearray.fromhex("d10c6ec5bae81b6cb9144de81037fa15")

In [29]:
iv_int = int.from_bytes(iv, "big")
print("Using block size of {} bytes".format(AES.block_size))
ctr = Counter.new(AES.block_size * 8, initial_value=iv_int)

Using block size of 16 bytes


In [30]:
cipher_enc = AES.new(encryption_key, AES.MODE_CTR, counter=ctr)

In [31]:
ciphertext = cipher_enc.encrypt(private_key)

In [32]:
print("Ciphertext is {} and of length {} bytes".format(ciphertext.hex(), len(ciphertext)))

Ciphertext is b4c5ccda033b7521099c6e49d46135fe047445a13827c4c6700d3dd90ea5aff3 and of length 32 bytes


In [33]:
cipher_dec = AES.new(encryption_key, AES.MODE_CTR, counter=ctr)

In [34]:
decrypted_key = cipher_dec.decrypt(ciphertext)

In [35]:
print("Decrypted private key is {} and of length {} bytes".format(decrypted_key.hex(), len(decrypted_key)))

Decrypted private key is acaf3289d7b601cbd114fb36c4d29c85bbfd5e133f14cb355c3fd8d99367964f and of length 32 bytes


In [36]:
# Finally create the MAC - Message Authentication Code

In [37]:
keccak_mac = keccak.new(digest_bits=256)
mac = keccak_mac.update(validation_key+ciphertext).digest()

In [38]:
print("MAC is {}".format(mac.hex()))

MAC is 7fd3326e8fd7325b7e95477bf7b04532bea423b0d2aa82952ecd8bb7ee368c85


In [39]:
# Lastly, we need to create the 20 byte Ethereum address

In [40]:
# First, we derive the public key from the private key. 
# It is worth noting that the public key is 64 bytes long (as opposed to a private key of 'only' 32 bytes)

In [41]:
public_key = keys.PrivateKey(private_key).public_key.to_bytes()

In [42]:
print("Public key is {} and of length {} bytes".format(public_key.hex(), len(public_key)))

Public key is 0bc544ce733e9ba5bd6d98b9ec4076479d3693ff29eed32dc7435ab77651ccae28833cac68200506d29f7752cc191fd32fc43b89264d8ab8ae141e5bfb0d76b6 and of length 64 bytes


In [43]:
# Finally, we construct the address from the last 20 bytes of keccak256 hash of the public key

In [44]:
keccak_address = keccak.new(digest_bits=256)
address = keccak_address.update(public_key).digest()[-20:]

In [45]:
print("The Ethereum public address is {} and is {} bytes long". format(address.hex(), len(address)))

The Ethereum public address is 046765101f9e8930eaf34664f8f8461d6fbd7e86 and is 20 bytes long


In [46]:
# Putting it all together

In [47]:
key_id = str(uuid.uuid4())
keystore = {
    "address": address.hex(),
    "crypto": {
        "cipher": "aes-128-ctr",
        "ciphertext": ciphertext.hex(),
        "cipherparams": {
            "iv": iv.hex()
        },
        "kdf": "scrypt",
        "kdfparams": {
            "dklen": 32,
            "n": number_of_rounds,
            "p": 1,
            "r": 8,
            "salt": salt.hex()
        },
        "mac": mac.hex()
    },
    "id": key_id,
    "version": 3
}

In [48]:
with open('keystore.json', 'w+') as f:
    json.dump(keystore, f, indent=2)